In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import pickle
import os
import numpy as np

# 定义VGG模型
class VGG(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 8 * 8, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

class CIFAR10Dataset(Dataset):
    def __init__(self, root, train = True):
        data_batch_list = []
        target_list = []
        for file in os.listdir(root):
            filename = os.path.join(root, file)
            with open(filename, 'rb') as fo:
                data_batch = pickle.load(fo, encoding='bytes')
                data_batch_list.append(data_batch[b'data'])
                target_list.extend(data_batch[b'labels'])
            
        self.data = np.vstack(data_batch_list).reshape(-1, 3, 32, 32)
        # self.data = self.data.transpose((0, 2, 3, 1))
        self.num_samples = self.data.shape[0]
        self.targets = target_list
        
    def __getitem__(self, idx):
        return torch.FloatTensor(self.data[idx]), self.targets[idx]
        
    def __len__(self):
        return self.num_samples
        
        
# 数据预处理和加载
#transform_train = transforms.Compose([
#    transforms.RandomCrop(32, padding=4),
#    transforms.RandomHorizontalFlip(),
#    transforms.ToTensor(),
#    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#])

#transform_test = transforms.Compose([
#    transforms.ToTensor(),
#    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#])
#trainset = CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainset = CIFAR10Dataset(root='./cifar_10/train', train=True)
testset = CIFAR10Dataset(root='./cifar_10/test', train=False)
#trainset = CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

#testset = CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# 初始化模型、损失函数和优化器
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device:", device)

model = VGG()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # 每100个小批量打印一次损失
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

# 测试模型
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the test images: %d %%' % (
    100 * correct / total))

Device: cuda:0
[1,   100] loss: 2.327
[1,   200] loss: 2.074
[1,   300] loss: 1.805
[1,   400] loss: 1.704
[1,   500] loss: 1.628
[1,   600] loss: 1.581
[1,   700] loss: 1.551
[2,   100] loss: 1.423
[2,   200] loss: 1.429
[2,   300] loss: 1.378
[2,   400] loss: 1.365
[2,   500] loss: 1.369
[2,   600] loss: 1.311
[2,   700] loss: 1.291
[3,   100] loss: 1.206
[3,   200] loss: 1.176
[3,   300] loss: 1.206
[3,   400] loss: 1.159
[3,   500] loss: 1.166
[3,   600] loss: 1.141
[3,   700] loss: 1.149
[4,   100] loss: 1.037
[4,   200] loss: 1.061
[4,   300] loss: 1.087
[4,   400] loss: 1.036
[4,   500] loss: 1.029
[4,   600] loss: 1.050
[4,   700] loss: 1.067
[5,   100] loss: 0.966
[5,   200] loss: 0.966
[5,   300] loss: 0.952
[5,   400] loss: 0.966
[5,   500] loss: 0.984
[5,   600] loss: 0.972
[5,   700] loss: 0.975
[6,   100] loss: 0.917
[6,   200] loss: 0.922
[6,   300] loss: 0.897
[6,   400] loss: 0.905
[6,   500] loss: 0.909
[6,   600] loss: 0.913
[6,   700] loss: 0.916
[7,   100] loss: 0.